In [1]:
pip install transformers


     |████████████████████████████████| 890kB 3.3MB/s 
     |████████████████████████████████| 890kB 12.2MB/s 
     |████████████████████████████████| 1.1MB 24.4MB/s 
     |████████████████████████████████| 3.0MB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=85dd9245f1fc9161f60003f9eba98ac910ef554857602d45e2e63d318576d0fc
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/German_train.csv')
# data = data[['text', 'task2']]
data.head()

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1131117000279961600,"Lehrstück auch, wie in der linken Jammerfemini...",HOF,NONE,hasoc_2020_de_2440
2,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
3,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
4,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559


In [4]:
data1=pd.read_excel("/content/drive/My Drive/MinorProject2/BERT2/german_train_new.xlsx")
# data1=data1[['text','task2']]
data1.head()

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
2,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
3,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559
4,1128743783393312768,RT @technosteron: leute die 'boar' schreiben l...,HOF,PRFN,hasoc_2020_de_1969


In [5]:
data['task2'].value_counts()

NONE    1860
PRFN     364
OFFN     126
HATE     102
Name: task2, dtype: int64

In [6]:
data1['task2'].value_counts()

NONE    1700
PRFN     387
HATE     146
OFFN     140
Name: task2, dtype: int64

In [7]:

data3 = data1[data1['task2']=='OFFN']
data3

,tweet_id,text,task1,task2,ID
15,1129884743137398784,"RT @johannesgrunert: Armselig, peinlich und du...",HOF,OFFN,hasoc_2020_de_1996
20,1131180745324728320,@KONRAD1000 @LVZ Wenn man Ihnen eine Körperver...,HOF,OFFN,hasoc_2020_de_2683
45,1129130045224685572,Schlagabtausch im ZDF..... regierungsgesteuer...,HOF,OFFN,hasoc_2020_de_1266
53,1125431663163531264,hab glaube echt gerade den größten hurensohn b...,HOF,OFFN,hasoc_2020_de_1305
80,1133436311979868161,@Tlaytweet hilfe diese frau was befindet sich ...,HOF,OFFN,hasoc_2020_de_2052
...,...,...,...,...,...
2322,1124583923013627904,@subversivaktiv @erkanallesffm Geh zurück ins ...,HOF,OFFN,hasoc_2020_de_240
2342,1129718078282117121,@ZeitW @welt Weil auch jeder ein hoher Politik...,HOF,OFFN,hasoc_2020_de_2963
2344,1134068150318878720,@wodojiban @princhillmaru9 😍🥰😘😜Der Schwanz ist...,HOF,OFFN,hasoc_2020_de_1328
2346,1124957329328357378,ich breche meine fingernägel an deinem rücken ab,HOF,OFFN,hasoc_2020_de_2995


In [8]:
data4=data1[data1['task2']=='HATE']
data4

,tweet_id,text,task1,task2,ID
27,1127483176929116160,"@SatireUNfreiesD So ist es, der Islam ist kein...",HOF,HATE,hasoc_2020_de_3043
38,1127239156508311557,"@Hartes_Geld #Nazis waren schon immer #Linke, ...",HOF,HATE,hasoc_2020_de_548
42,1130112447690530816,RT @braindead_xy: @tagesschau Deutsche Regieru...,HOF,HATE,hasoc_2020_de_2368
70,1132931582032076800,die arroganz der leute aus den westlichen bund...,HOF,HATE,hasoc_2020_de_1886
72,1125147385791942656,RT @burger_ein: Radikale Moslems und nationali...,HOF,HATE,hasoc_2020_de_2217
...,...,...,...,...,...
2324,1134395448637173761,RT @alfunterwegs70: Sie belügen euch ganz öffe...,HOF,HATE,hasoc_2020_de_1409
2329,1123614787143962626,"@SandraLustig13 Mannomann, dumme, verwöhnte, g...",HOF,HATE,hasoc_2020_de_3096
2340,1131102202762805248,RT @helllud123: @ZDFheute @sven_giegold @Joerg...,HOF,HATE,hasoc_2020_de_177
2348,1124930687084187649,RT @den_tyske: ... gibt es wirklich noch Idiot...,HOF,HATE,hasoc_2020_de_551


In [10]:
data=pd.concat([data,data3,data4])
data

,tweet_id,text,task1,task2,ID
0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1131117000279961600,"Lehrstück auch, wie in der linken Jammerfemini...",HOF,NONE,hasoc_2020_de_2440
2,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
3,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
4,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559
...,...,...,...,...,...
2324,1134395448637173761,RT @alfunterwegs70: Sie belügen euch ganz öffe...,HOF,HATE,hasoc_2020_de_1409
2329,1123614787143962626,"@SandraLustig13 Mannomann, dumme, verwöhnte, g...",HOF,HATE,hasoc_2020_de_3096
2340,1131102202762805248,RT @helllud123: @ZDFheute @sven_giegold @Joerg...,HOF,HATE,hasoc_2020_de_177
2348,1124930687084187649,RT @den_tyske: ... gibt es wirklich noch Idiot...,HOF,HATE,hasoc_2020_de_551


In [11]:
data.to_csv('new_german.csv')


In [12]:
df=pd.read_csv('/content/new_german.csv')
df

,Unnamed: 0,tweet_id,text,task1,task2,ID
0,0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1,1131117000279961600,"Lehrstück auch, wie in der linken Jammerfemini...",HOF,NONE,hasoc_2020_de_2440
2,2,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
3,3,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
4,4,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559
...,...,...,...,...,...,...
3019,2324,1134395448637173761,RT @alfunterwegs70: Sie belügen euch ganz öffe...,HOF,HATE,hasoc_2020_de_1409
3020,2329,1123614787143962626,"@SandraLustig13 Mannomann, dumme, verwöhnte, g...",HOF,HATE,hasoc_2020_de_3096
3021,2340,1131102202762805248,RT @helllud123: @ZDFheute @sven_giegold @Joerg...,HOF,HATE,hasoc_2020_de_177
3022,2348,1124930687084187649,RT @den_tyske: ... gibt es wirklich noch Idiot...,HOF,HATE,hasoc_2020_de_551


In [13]:
data = data.sample(frac = 1)
data.head()

,tweet_id,text,task1,task2,ID
1009,1127992893917016064,@dopetobetattoo @tagesschau doch,NOT,NONE,hasoc_2020_de_988
2344,1134068150318878720,@wodojiban @princhillmaru9 😍🥰😘😜Der Schwanz ist...,HOF,OFFN,hasoc_2020_de_1328
55,1125778708260896770,RT @tagesschau: EVP-Spitzenkandidat Weber will...,NOT,NONE,hasoc_2020_de_1254
1947,1126746606810779648,RT @psychomolehappy: die die jetzt schreiben i...,NOT,PRFN,hasoc_2020_de_2530
1574,1125056482649812994,RT @kimmathei: @Taddl Reiß dich zusammen du du...,HOF,OFFN,hasoc_2020_de_3175


In [14]:
data.shape

(3024, 5)

In [15]:
data=data[['text','task2']]
data

,text,task2
1009,@dopetobetattoo @tagesschau doch,NONE
2344,@wodojiban @princhillmaru9 😍🥰😘😜Der Schwanz ist...,OFFN
55,RT @tagesschau: EVP-Spitzenkandidat Weber will...,NONE
1947,RT @psychomolehappy: die die jetzt schreiben i...,PRFN
1574,RT @kimmathei: @Taddl Reiß dich zusammen du du...,OFFN
...,...,...
2340,"@CodingCookey But hey, ich muss ja jetzt eh ni...",NONE
278,Gunlike ist gerade von schwule hurensohn zu ei...,OFFN
2182,@HeikoMaas Sehr geehrter Herr #HeikoMaas warum...,NONE
1946,@Borussen_Tom09 @EAFussball Die wahren Idioten...,NONE


In [16]:
data['task2'] = pd.Categorical(data['task2'])
data['task2'] = data['task2'].cat.codes
data.head()

,text,task2
1009,@dopetobetattoo @tagesschau doch,1
2344,@wodojiban @princhillmaru9 😍🥰😘😜Der Schwanz ist...,2
55,RT @tagesschau: EVP-Spitzenkandidat Weber will...,1
1947,RT @psychomolehappy: die die jetzt schreiben i...,3
1574,RT @kimmathei: @Taddl Reiß dich zusammen du du...,2


In [17]:
data.shape

(3024, 2)

In [18]:
# df['task2'] = pd.Categorical(df['task2'])
# df['task2'] = df['task2'].cat.codes
# df.head()

In [19]:
data, data_test = train_test_split(data, test_size = 0.2)
data.shape,data_test.shape

((2419, 2), (605, 2))

In [20]:
data.task2.value_counts()

1    1497
0     320
2     315
3     287
Name: task2, dtype: int64

In [21]:
model_name = 'bert-base-uncased'
max_length = 64
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
task2 = Dense(units=len(data.task2.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='task2')(pooled_output)
outputs = {'task2': task2}
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()


Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 64)]              0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 64, 768), (None,  109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
task2 (Dense)                (None, 4)                 3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


In [23]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
loss = {'task2': CategoricalCrossentropy(from_logits = True)}
metric = {'task2': CategoricalAccuracy('accuracy')}

In [24]:
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
y_issue = to_categorical(data['task2'])

In [25]:
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'task2': y_issue},
    validation_split=0.2,
    batch_size=64,
    epochs=15)

Epoch 1/15
31/31 [==============================] - 33s 1s/step - loss: 1.0917 - accuracy: 0.6109 - val_loss: 1.0677 - val_accuracy: 0.6157
Epoch 2/15
31/31 [==============================] - 31s 987ms/step - loss: 1.0200 - accuracy: 0.6217 - val_loss: 0.8846 - val_accuracy: 0.6674
Epoch 3/15
31/31 [==============================] - 31s 990ms/step - loss: 0.8407 - accuracy: 0.6698 - val_loss: 0.8614 - val_accuracy: 0.6694
Epoch 4/15
31/31 [==============================] - 31s 996ms/step - loss: 0.6869 - accuracy: 0.7276 - val_loss: 0.7385 - val_accuracy: 0.7169
Epoch 5/15
31/31 [==============================] - 31s 1s/step - loss: 0.4785 - accuracy: 0.8346 - val_loss: 0.6761 - val_accuracy: 0.7665
Epoch 6/15
31/31 [==============================] - 31s 999ms/step - loss: 0.3209 - accuracy: 0.9003 - val_loss: 0.6674 - val_accuracy: 0.7727
Epoch 7/15
31/31 [==============================] - 31s 999ms/step - loss: 0.2136 - accuracy: 0.9385 - val_loss: 0.6238 - val_accuracy: 0.8202
Epoch

In [26]:
test_y_issue = to_categorical(data_test['task2'])
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'task2': test_y_issue}
)

19/19 [==============================] - 3s 170ms/step - loss: 0.8799 - accuracy: 0.7818


In [27]:
test_x

{'input_ids': <tf.Tensor: shape=(605, 64), dtype=int32, numpy=
array([[  101,  1030,  9253, ...,     0,     0,     0],
       [  101,  8040,  3270, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0],
       ...,
       [  101, 19387,  1030, ...,     0,     0,     0],
       [  101, 19387,  1030, ...,     0,     0,     0],
       [  101, 17924, 12722, ...,     0,     0,     0]], dtype=int32)>}

In [28]:
preds = model.predict(test_x['input_ids'])

In [29]:
preds=preds['task2'].argmax(axis=-1)

In [30]:
preds

array([0, 3, 0, 1, 1, 1, 3, 1, 1, 0, 2, 2, 1, 0, 3, 1, 1, 1, 1, 1, 1, 0,
       2, 2, 2, 1, 1, 2, 0, 0, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 0, 2, 2, 3, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 0, 1,
       0, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 3, 1, 1, 0, 1, 0,
       1, 0, 3, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 1, 1, 0, 1, 2, 2, 3, 1, 1,
       1, 1, 1, 0, 1, 2, 2, 3, 1, 1, 1, 3, 1, 0, 2, 0, 3, 2, 2, 2, 3, 2,
       1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 0, 1, 2,
       1, 0, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 2, 3, 2, 3,
       1, 1, 3, 1, 1, 2, 1, 3, 2, 0, 1, 1, 1, 1, 2, 0, 0, 3, 1, 2, 3, 1,
       1, 3, 2, 1, 1, 3, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 2, 1, 1, 1, 0, 1,
       1, 1, 0, 2, 3, 2, 0, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 3, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 1, 1, 1, 0, 0, 0,
       2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 3, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 2, 2, 1, 3, 2, 3, 1, 1, 0, 1, 2, 2, 3,

In [31]:
test_y_issue=test_y_issue.argmax(axis=-1)

In [32]:
test_y_issue

array([1, 3, 0, 1, 0, 1, 3, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 1, 0,
       2, 2, 3, 3, 1, 2, 0, 1, 1, 0, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 1, 2, 2, 3, 3, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 3, 1, 3, 0, 3,
       0, 1, 1, 1, 2, 0, 0, 3, 1, 1, 1, 2, 1, 1, 0, 0, 3, 0, 1, 1, 1, 1,
       1, 0, 3, 0, 1, 1, 2, 2, 1, 1, 3, 1, 3, 1, 1, 0, 1, 2, 2, 3, 1, 1,
       1, 1, 1, 2, 1, 2, 2, 3, 1, 3, 1, 3, 1, 0, 2, 1, 3, 2, 2, 2, 2, 2,
       1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 0, 1, 2,
       3, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 2, 3,
       1, 3, 3, 0, 1, 1, 1, 3, 2, 0, 1, 1, 1, 1, 2, 0, 1, 3, 1, 3, 3, 1,
       0, 3, 1, 1, 1, 3, 1, 1, 1, 0, 1, 1, 1, 1, 3, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 2, 2, 2, 1, 3, 0, 1, 3, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 3, 3, 1, 0, 0, 0, 1, 1,
       1, 1, 2, 2, 3, 3, 2, 3, 1, 2, 0, 1, 2, 3, 1,

In [33]:
from sklearn.metrics import classification_report
print(classification_report(test_y_issue,preds))

              precision    recall  f1-score   support

           0       0.59      0.64      0.61        74
           1       0.84      0.85      0.85       363
           2       0.79      0.85      0.81        91
           3       0.65      0.52      0.58        77

    accuracy                           0.78       605
   macro avg       0.72      0.71      0.71       605
weighted avg       0.78      0.78      0.78       605

